In [1]:
import networkx as nx
from random import randrange
from math import sqrt, log, inf
import numpy as np
from copy import deepcopy
import pandas
import csv

import sys, os
working_directory = 'C:\\Users\\ronal\\Documents\\GitHub\\decision-making\\'
# Append path where graph_builder.py is saved
sys.path.append(working_directory + 'graph_builder')
from graph_builder import *

file_names = os.listdir(working_directory + 'path_analysis\\treebuilderUpdated')
graphs = {}
for file in file_names:
    graphs[file.replace('.csv', '')] = build_graph(
        working_directory + 'path_analysis\\treebuilderUpdated\\' + file)

In [2]:
from mcts import UCTSearch

In [ ]:
{node: my_search.normalize_value(1 / value, test_values) for node, value in choices_test.items()}

## Add MCTS choice values to probabilities_to_values.csv

In [3]:
def parse_prob_data(data_file):
    prob_data_frame = pd.read_csv(data_file, sep='\t')
    parsed_data = []
    
    for row in prob_data_frame.itertuples(index=False):
        parsed_data.append(list(row[:7]))
    
    return parsed_data

In [4]:
parsed_data = parse_prob_data('..//path_analysis/probabilities_to_values.csv')

In [7]:
def analyze_prob_data(parsed_data):
    input_data = deepcopy(parsed_data)
    output_data = []
    
    for row in input_data: # Analyze path that each subject followed
        MCTS_value_choices = ['None', 'None', 'None', 'None']
        graph_name = row[0]
        input_path = row[6].split(',')
        
        search_algorithm = UCTSearch(graphs[graph_name])
        value_choices_dict = search_algorithm.generate_target_path(input_path)[-1][-1]
        
        ## Invert values again. We want better values to be lower
        value_list = [1 / value for value in value_choices_dict.values()]
        
        ## Normalize the values
        corrected_value_choices_dict = {node: search_algorithm.normalize_value(1 / value, value_list) for node, value in value_choices_dict.items()}
        
        for successor in value_choices_dict:
            index = search_algorithm.tree.nodes[successor]['choice_direction']
            
            MCTS_value_choices[index] = corrected_value_choices_dict[successor]
        
        row.extend(MCTS_value_choices)
        
        output_data.append(row)

    print(f'{len(input_data)} paths analyzed')
    
    return output_data

In [8]:
output_data = analyze_prob_data(parsed_data)

136 paths analyzed


In [14]:
# Export probability analysis as csv
def export_prob_results(output_data):
    column_titles = ['world', 'path', 'probL', 'probR', 'probU', 'probD', 
                     'subject_node_path', 'MCTS_value_L',
                    'MCTS_value_R', 'MCTS_value_U', 'MCTS_value_D']

    with open('probabilities_to_values_MCTS.csv', 'w', newline='') as file:
        file_writer = csv.writer(file, delimiter='\t')
        file_writer.writerow(column_titles)

        for row in output_data:
            file_writer.writerow(row)

In [13]:
export_prob_results(output_data)